<a href="https://colab.research.google.com/github/AK18k/lora/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AK18k/lora

Cloning into 'lora'...
remote: Enumerating objects: 1405, done.
remote: Counting objects: 100% (1405/1405), done.
remote: Compressing objects: 100% (999/999), done.
remote: Total 1405 (delta 389), reused 1377 (delta 371), pack-reused 0
Receiving objects: 100% (1405/1405), 23.87 MiB | 16.50 MiB/s, done.
Resolving deltas: 100% (389/389), done.


In [2]:
!pip install progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=10a4294f0d7e5d71323e9331172ed093d121b977e4bb8b7fb266b2f6531f68b0
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [3]:
# !cd ./lora/examples/NLG
# !ls '*.sh'
!bash ./lora/examples/NLG/download_pretrained_checkpoints.sh

downloading pretrained model checkpoints...
--2023-06-03 15:08:14--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.128.136, 52.216.211.248, 52.216.109.133, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.128.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548118077 (523M) [application/octet-stream]
Saving to: ‘gpt2-pytorch_model.bin’

gpt2-pytorch_model. 100%[===================>] 522.73M  33.2MB/s    in 17s     

2023-06-03 15:08:31 (31.4 MB/s) - ‘gpt2-pytorch_model.bin’ saved [548118077/548118077]

--2023-06-03 15:08:31--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-medium-pytorch_model.bin
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.138.101, 52.217.233.176, 52.217.75.150, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.138.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1520013706 (1.4G) [a

In [4]:
!bash ./lora/examples/NLG/create_datasets.sh

creating e2e datasets...
train...
test...
valid...
creating webnlg datasets...
train...
cate Airport 1090
cate Astronaut 530
cate Building 972
cate City 243
cate ComicsCharacter 285
cate Food 1424
cate Monument 267
cate SportsTeam 786
cate University 406
cate WrittenWork 937
test...
cate Airport 136
cate Monument 33
cate Building 120
cate WrittenWork 116
cate SportsTeam 98
cate University 51
cate Astronaut 66
cate ComicsCharacter 35
cate City 139
cate Food 177
cate Politician 209
cate MeanOfTransportation 198
cate Athlete 159
cate Artist 213
cate CelestialBody 112
valid...
cate Airport 136
cate Astronaut 67
cate Building 123
cate City 31
cate ComicsCharacter 37
cate Food 178
cate Monument 32
cate SportsTeam 99
cate University 51
cate WrittenWork 118
creating dart datasets...
train...
unique source is 30526
test...
unique source is 6959
valid...
unique source is 2768
script complete!


In [5]:
!bash /content/lora/examples/NLG/eval/download_evalscript.sh


installing evaluation dependencies
downloading e2e-metrics...
Cloning into 'e2e'...
remote: Enumerating objects: 909, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 909 (delta 0), reused 1 (delta 0), pack-reused 906
Receiving objects: 100% (909/909), 106.78 MiB | 14.45 MiB/s, done.
Resolving deltas: 100% (492/492), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
downloading GenerationEval for webnlg and dart...
Cloning into 'GenerationEval'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 82 (delta 36), reused 51 (delta 15), pack-reused 0
Unpacking objects: 100% (82/82), 221.17 KiB | 5.27 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.5 MB/s et

In [6]:
import sys
sys.path.append('/content/lora/')
sys.path.append('/content/lora/examples/NLG/src')

In [7]:
!pwd
!torchrun --nproc_per_node=1 /content/lora/examples/NLG/src/gpt2_ft.py \
    --train_data /content/lora/examples/NLG/data/e2e/train.jsonl \
    --valid_data /content/lora/examples/NLG/data/e2e/valid.jsonl \
    --train_batch_size 8 \
    --grad_acc 1 \
    --valid_batch_size 4 \
    --seq_len 512 \
    --model_card gpt2.md \
    --init_checkpoint ./pretrained_checkpoints/gpt2-medium-pytorch_model.bin \
    --platform local \
    --clip 0.0 \
    --lr 0.0002 \
    --weight_decay 0.01 \
    --correct_bias \
    --adam_beta2 0.999 \
    --scheduler linear \
    --warmup_step 500 \
    --max_epoch 5 \
    --save_interval 1000 \
    --lora_dim 4 \
    --lora_alpha 32 \
    --lora_dropout 0.1 \
    --label_smooth 0.1 \
    --work_dir ./trained_models/GPT2_M/e2e \
    --random_seed 110

/content
myrank: 0 local_rank: 0 device_count: 1 world_size: 1
        - platform : local
        - local_rank : 0
        - rank : 0
        - device : cuda:0
        - world_size : 1
        - random_seed : 110
        - lr : 0.0002
        - weight_decay : 0.01
        - correct_bias : True
        - adam_epislon : 1e-06
        - no_decay_bias : False
        - adam_beta1 : 0.9
        - adam_beta2 : 0.999
        - scheduler : linear
        - max_step : None
        - max_epoch : 5
        - warmup_step : 500
        - i_steps : 0
        - i_lrs : 0.00025
        - train_data : ./data/e2e/train.jsonl
        - valid_data : ./data/e2e/valid.jsonl
        - train_batch_size : 8
        - valid_batch_size : 4
        - grad_acc : 1
        - clip : 0.0
        - seq_len : 512
        - model_card : gpt2.md
        - init_checkpoint : ./pretrained_checkpoints/gpt2-medium-pytorch_model.bin
        - fp16 : False
        - log_interval : 100
        - eval_interval : 2000
        - sa